In [ ]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt

from pca.pcaIO import load_pca
from database.query import get_protdef
from utils.output_namespace import aligned_pdbfile, init_gro
from plot.plot_utilities import hist2d, edgeformat

# Load the PC values computed and stored

In [ ]:
pca_data = load_pca('6msm_tmpc/all_tmpc.stride1.realign')
pca_df = pca_data.pca_df(n_pcs=2)
xyzcenter = pca_data.xyz_center
eigenvec = pca_data.eigenvec
atomselect = pca_data.atomselect
atomstride = pca_data.atomstride

%store pca_df
pca_df

In [ ]:
domain_dict = get_protdef(1)
tm_helices = [f'TM{n}' for n in range(1,12+1)]

pdbstruc = md.load(aligned_pdbfile('6msm'))
pdbstruc = md.load(init_gro(1))
tmselect = pdbstruc.top.select(atomselect)[::atomstride]
pdb_tmxyz = pdbstruc.xyz[0,tmselect,:].flatten()
    
pdb_tmproject = np.dot(eigenvec[:2,:], (pdb_tmxyz*10 - xyzcenter))

# Plot (PC1,PC2)

In [ ]:
# Quick contour view
fig, axs = plt.subplots()

xrange = [-60,60]
yrange = [-50,50]

hist = hist2d(*pca_df[['pc1', 'pc2']].values.T, bins=60, range=[xrange, yrange])
cbar = hist.dens2d_preset2(axs, nlevels=10, cbar_show=True, level_multiplier_exponent=5, lw=0.3)
axs.set_aspect('equal', adjustable='box', anchor='C')
axs.set_xlabel(r"PC1 [$\mathrm{\AA}$]", fontsize=16)
axs.set_ylabel(r"PC2 [$\mathrm{\AA}$]", fontsize=16)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.grid(True, linestyle='--')

axs.scatter([pdb_tmproject[0]], [pdb_tmproject[1]], marker='x', color='blue', s=100)

cbar.ax.set_yticklabels(np.round(cbar.get_ticks() / np.max(cbar.get_ticks()), decimals=1)*10, size=12)
cbar.ax.set_ylabel('prob. density [A.U.]', fontsize=16)

# savefig('updated_pca_contour.pdf')

# Make porcupine plot

In [ ]:
def porcupine(xyz2drawfrom, eigenvec, scale=20, script='porcupine.tcl'):
    # Reshape eigenvec to (n_atoms, 3)
    vectors2draw = eigenvec.reshape(len(eigenvec)//3, 3)
    xyz2drawto = xyz2drawfrom + vectors2draw * scale
    
    # Write the porcupine plot drawing script for VMD
    with open(script, "w") as file:
        for begin, end in zip(xyz2drawfrom, xyz2drawto):
            file.write(f"graphics top cone {{{' '.join(map(str, begin))}}} {{{' '.join(map(str, end))}}} radius [expr 0.2 * 1.7] resolution 10\n")

# PCA of all TM-helices: atomstride=1; demo

In [ ]:
from pca.pca_analysis_workflow import analyze_pca

In [ ]:
%store -r traj_ids

name = 'alltm_CApc'
datadir = '6msm_tmpc/all_tmpc.stride1.realign'
n_pcs = 2

N = 4
xrange = [-70,70]
yrange = [-55,55]

main = analyze_pca(datadir, n_pcs, traj_ids)

fig, axs = plt.subplots(1, 2, figsize=(12,4), sharey=True, gridspec_kw={'wspace': 0.15})

for ax in axs:
    edgeformat(ax)

main.pca_data.plot_explained_variance(axs[0])
main.pca_data.plot_cumulative_variance(axs[1])
# _ = main.variance_plots()
# savefig("pca_variance.pdf")

_ = main.state_clustering(N, xrange, yrange)
